<a href="https://colab.research.google.com/github/haruyoshikawabe/accounting-automation-python/blob/main/%E5%8B%98%E5%AE%9A%E7%A7%91%E7%9B%AE%E5%88%A4%E5%AE%9A%E3%83%84%E3%83%BC%E3%83%AB%E6%94%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from collections import Counter
import re
from flask import Flask, request, jsonify, render_template_string
from threading import Thread
import os

# Flaskアプリの初期化
app = Flask(__name__)

class AccountSuggestionEngine:
    """勘定科目推定エンジン"""

    def __init__(self, journal_csv_path='journal.csv'):
        self.journal_df = None
        self.load_journal_data(journal_csv_path)

    def load_journal_data(self, csv_path):
        """過去仕訳データの読み込み"""
        try:
            self.journal_df = pd.read_csv(csv_path, encoding='utf-8-sig')
            print(f"仕訳データ読み込み完了: {len(self.journal_df)}件")
        except FileNotFoundError:
            print(f"警告: {csv_path} が見つかりません。サンプルデータを作成します。")
            self.create_sample_data(csv_path)
            self.journal_df = pd.read_csv(csv_path, encoding='utf-8-sig')

    def create_sample_data(self, csv_path):
        """サンプル仕訳データの作成"""
        sample_data = {
            '日付': ['2024-01-10', '2024-01-15', '2024-01-20', '2024-01-25', '2024-02-01',
                    '2024-02-05', '2024-02-10', '2024-02-15', '2024-02-20', '2024-02-25'],
            '取引先': ['株式会社ABC', '株式会社ABC', '山田商店', '株式会社XYZ', '株式会社ABC',
                     '山田商店', '田中工務店', '株式会社XYZ', '株式会社ABC', '田中工務店'],
            '摘要': ['事務用品購入', '文房具購入', '広告宣伝費 チラシ作成', '交通費 出張', '消耗品購入',
                   '広告費 WEB広告', '修繕費 事務所', '旅費交通費 新幹線', '備品購入', '修繕費 看板'],
            '勘定科目': ['消耗品費', '消耗品費', '広告宣伝費', '旅費交通費', '消耗品費',
                      '広告宣伝費', '修繕費', '旅費交通費', '消耗品費', '修繕費'],
            '金額': [5000, 3000, 50000, 15000, 8000, 30000, 80000, 20000, 12000, 45000]
        }
        df = pd.DataFrame(sample_data)
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        print(f"サンプルデータを作成しました: {csv_path}")

    def suggest_accounts(self, vendor, description, top_k=3):
        """勘定科目候補を推定"""
        if self.journal_df is None or len(self.journal_df) == 0:
            return []

        candidates = {}

        # 1. 取引先別頻度スコア
        vendor_matches = self.journal_df[self.journal_df['取引先'] == vendor]
        if len(vendor_matches) > 0:
            account_freq = Counter(vendor_matches['勘定科目'])
            total = len(vendor_matches)
            for account, count in account_freq.items():
                if account not in candidates:
                    candidates[account] = {'score': 0, 'reasons': []}
                vendor_score = (count / total) * 40  # 最大40点
                candidates[account]['score'] += vendor_score
                candidates[account]['reasons'].append(
                    f"取引先「{vendor}」での使用実績: {count}/{total}回 ({count/total*100:.1f}%)"
                )

        # 2. キーワード一致スコア
        keywords_map = {
            '消耗品費': ['消耗品', '文房具', '事務用品', '備品'],
            '広告宣伝費': ['広告', '宣伝', 'チラシ', 'WEB', 'ポスター'],
            '旅費交通費': ['交通費', '旅費', '出張', '新幹線', 'タクシー', '電車'],
            '修繕費': ['修繕', '修理', '補修'],
            '通信費': ['通信', '電話', 'インターネット', '携帯'],
            '水道光熱費': ['電気', 'ガス', '水道'],
            '接待交際費': ['接待', '交際', '会食', '贈答'],
        }

        description_lower = description.lower()
        for account, keywords in keywords_map.items():
            for keyword in keywords:
                if keyword in description:
                    if account not in candidates:
                        candidates[account] = {'score': 0, 'reasons': []}
                    candidates[account]['score'] += 30  # キーワード一致で30点
                    candidates[account]['reasons'].append(
                        f"キーワード「{keyword}」が摘要に含まれています"
                    )
                    break  # 同じ勘定科目で複数キーワードが一致しても加点は1回のみ

        # 3. 摘要類似度スコア（簡易版: 部分一致）
        for _, row in self.journal_df.iterrows():
            past_desc = str(row['摘要'])
            account = row['勘定科目']

            # 共通する単語をカウント（簡易的な類似度）
            desc_words = set(re.findall(r'\w+', description))
            past_words = set(re.findall(r'\w+', past_desc))
            common_words = desc_words & past_words

            if len(common_words) > 0:
                similarity_score = len(common_words) / max(len(desc_words), len(past_words)) * 30
                if account not in candidates:
                    candidates[account] = {'score': 0, 'reasons': []}
                # 類似度スコアは最大値を採用
                current_sim_score = 0
                for reason in candidates[account]['reasons']:
                    if '類似する過去仕訳' in reason:
                        current_sim_score = float(re.search(r'類似度(\d+\.?\d*)%', reason).group(1))

                if similarity_score > current_sim_score:
                    # 既存の類似度理由を削除
                    candidates[account]['reasons'] = [
                        r for r in candidates[account]['reasons']
                        if '類似する過去仕訳' not in r
                    ]
                    candidates[account]['score'] = candidates[account]['score'] - current_sim_score + similarity_score
                    candidates[account]['reasons'].append(
                        f"類似する過去仕訳「{past_desc}」(類似度{similarity_score:.1f}%)"
                    )

        # スコア順にソート
        sorted_candidates = sorted(
            candidates.items(),
            key=lambda x: x[1]['score'],
            reverse=True
        )[:top_k]

        # 結果を整形
        results = []
        for account, data in sorted_candidates:
            results.append({
                '勘定科目': account,
                'スコア': round(data['score'], 1),
                '理由': data['reasons']
            })

        return results

# グローバル変数でエンジンを保持
engine = None

def save_review_log(date, vendor, description, selected_account, memo):
    """選択された科目とメモを保存"""
    log_path = 'review_log.csv'

    log_data = {
        '日時': [datetime.now().strftime('%Y-%m-%d %H:%M:%S')],
        '日付': [date],
        '取引先': [vendor],
        '摘要': [description],
        '選択科目': [selected_account],
        'メモ': [memo]
    }

    df = pd.DataFrame(log_data)

    # ファイルが存在する場合は追記、存在しない場合は新規作成
    if os.path.exists(log_path):
        df.to_csv(log_path, mode='a', header=False, index=False, encoding='utf-8-sig')
    else:
        df.to_csv(log_path, index=False, encoding='utf-8-sig')

# HTMLテンプレート
HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>勘定科目判断支援ツール</title>
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }
        .container {
            max-width: 900px;
            margin: 0 auto;
            background: white;
            border-radius: 15px;
            box-shadow: 0 10px 40px rgba(0,0,0,0.2);
            padding: 40px;
        }
        h1 {
            color: #333;
            margin-bottom: 10px;
            font-size: 28px;
        }
        .subtitle {
            color: #666;
            margin-bottom: 30px;
            font-size: 14px;
        }
        .upload-section {
            background: #f8f9fa;
            border: 2px dashed #667eea;
            border-radius: 10px;
            padding: 25px;
            margin-bottom: 30px;
            text-align: center;
        }
        .upload-section h3 {
            color: #333;
            margin-bottom: 15px;
            font-size: 18px;
        }
        .file-input-wrapper {
            position: relative;
            display: inline-block;
            cursor: pointer;
            margin-top: 10px;
        }
        .file-input-wrapper input[type="file"] {
            position: absolute;
            opacity: 0;
            width: 100%;
            height: 100%;
            cursor: pointer;
        }
        .file-input-label {
            background: #667eea;
            color: white;
            padding: 12px 25px;
            border-radius: 8px;
            font-size: 15px;
            font-weight: 600;
            cursor: pointer;
            transition: all 0.3s;
            display: inline-block;
        }
        .file-input-label:hover {
            background: #5568d3;
            transform: translateY(-2px);
            box-shadow: 0 5px 15px rgba(102, 126, 234, 0.3);
        }
        .file-name {
            margin-top: 10px;
            color: #666;
            font-size: 14px;
        }
        .data-info {
            margin-top: 15px;
            padding: 12px;
            background: #e8eaf6;
            border-radius: 6px;
            font-size: 14px;
            color: #333;
        }
        .form-group {
            margin-bottom: 20px;
        }
        label {
            display: block;
            margin-bottom: 8px;
            color: #555;
            font-weight: 600;
            font-size: 14px;
        }
        input[type="text"], input[type="date"], textarea {
            width: 100%;
            padding: 12px;
            border: 2px solid #e0e0e0;
            border-radius: 8px;
            font-size: 15px;
            transition: border-color 0.3s;
        }
        input:focus, textarea:focus {
            outline: none;
            border-color: #667eea;
        }
        button {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 14px 30px;
            border: none;
            border-radius: 8px;
            font-size: 16px;
            font-weight: 600;
            cursor: pointer;
            transition: transform 0.2s, box-shadow 0.2s;
            width: 100%;
        }
        button:hover {
            transform: translateY(-2px);
            box-shadow: 0 5px 20px rgba(102, 126, 234, 0.4);
        }
        button:active {
            transform: translateY(0);
        }
        .results {
            margin-top: 30px;
            display: none;
        }
        .candidate {
            background: #f8f9fa;
            border: 2px solid #e0e0e0;
            border-radius: 10px;
            padding: 20px;
            margin-bottom: 15px;
            transition: all 0.3s;
            cursor: pointer;
        }
        .candidate:hover {
            border-color: #667eea;
            box-shadow: 0 3px 15px rgba(102, 126, 234, 0.2);
        }
        .candidate.selected {
            background: #e8eaf6;
            border-color: #667eea;
            border-width: 3px;
        }
        .candidate-header {
            display: flex;
            justify-content: space-between;
            align-items: center;
            margin-bottom: 12px;
        }
        .account-name {
            font-size: 20px;
            font-weight: 700;
            color: #333;
        }
        .score {
            background: #667eea;
            color: white;
            padding: 6px 14px;
            border-radius: 20px;
            font-size: 14px;
            font-weight: 600;
        }
        .reasons {
            color: #666;
            font-size: 14px;
            line-height: 1.6;
        }
        .reason-item {
            padding: 6px 0;
            padding-left: 20px;
            position: relative;
        }
        .reason-item:before {
            content: "•";
            position: absolute;
            left: 8px;
            color: #667eea;
            font-weight: bold;
        }
        .confirm-section {
            margin-top: 25px;
            padding: 25px;
            background: #f0f4ff;
            border-radius: 10px;
            display: none;
        }
        .memo-input {
            margin-top: 15px;
        }
        .confirm-btn {
            margin-top: 15px;
            background: #4caf50;
        }
        .confirm-btn:hover {
            box-shadow: 0 5px 20px rgba(76, 175, 80, 0.4);
        }
        .message {
            margin-top: 20px;
            padding: 15px;
            border-radius: 8px;
            display: none;
        }
        .message.success {
            background: #d4edda;
            color: #155724;
            border: 1px solid #c3e6cb;
        }
        .message.error {
            background: #f8d7da;
            color: #721c24;
            border: 1px solid #f5c6cb;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>📊 勘定科目判断支援ツール</h1>
        <p class="subtitle">取引情報を入力すると、過去の仕訳データをもとに勘定科目の候補を提示します</p>

        <!-- 過去仕訳データアップロードセクション -->
        <div class="upload-section">
            <h3>📁 過去仕訳データ (journal.csv)</h3>
            <p style="color: #666; font-size: 14px; margin-bottom: 10px;">
                CSVファイルをアップロードして学習データを更新できます
            </p>
            <div class="file-input-wrapper">
                <input type="file" id="journalFile" accept=".csv" onchange="uploadJournal()">
                <label class="file-input-label">📂 ファイルを選択</label>
            </div>
            <div id="fileName" class="file-name"></div>
            <div id="dataInfo" class="data-info" style="display: none;"></div>
        </div>

        <div class="form-group">
            <label for="date">日付</label>
            <input type="date" id="date" required>
        </div>

        <div class="form-group">
            <label for="vendor">取引先</label>
            <input type="text" id="vendor" placeholder="例: 株式会社ABC" required>
        </div>

        <div class="form-group">
            <label for="description">摘要</label>
            <input type="text" id="description" placeholder="例: 事務用品購入" required>
        </div>

        <button onclick="getSuggestions()">勘定科目を推定</button>

        <div id="results" class="results">
            <h2 style="margin-bottom: 20px; color: #333;">推定結果</h2>
            <div id="candidates"></div>
        </div>

        <div id="confirmSection" class="confirm-section">
            <h3 style="margin-bottom: 15px; color: #333;">選択科目の確認</h3>
            <p>選択科目: <strong id="selectedAccount"></strong></p>
            <div class="memo-input">
                <label for="memo">メモ（任意）</label>
                <textarea id="memo" rows="3" placeholder="判断の補足情報や注意点などを記入できます"></textarea>
            </div>
            <button class="confirm-btn" onclick="confirmSelection()">この科目で確定</button>
        </div>

        <div id="message" class="message"></div>
    </div>

    <script>
        let selectedAccount = null;

        // 今日の日付を設定
        document.getElementById('date').valueAsDate = new Date();

        // ページ読み込み時にデータ情報を取得
        window.onload = function() {
            fetchDataInfo();
        };

        function uploadJournal() {
            const fileInput = document.getElementById('journalFile');
            const file = fileInput.files[0];

            if (!file) return;

            const formData = new FormData();
            formData.append('file', file);

            document.getElementById('fileName').textContent = 'アップロード中...';

            fetch('/upload_journal', {
                method: 'POST',
                body: formData
            })
            .then(response => response.json())
            .then(data => {
                if (data.status === 'success') {
                    document.getElementById('fileName').textContent = '✓ ' + file.name + ' をアップロードしました';
                    document.getElementById('fileName').style.color = '#4caf50';
                    showMessage('仕訳データを更新しました!', 'success');
                    fetchDataInfo();
                } else {
                    document.getElementById('fileName').textContent = '✗ アップロード失敗';
                    document.getElementById('fileName').style.color = '#f44336';
                    showMessage('エラー: ' + data.message, 'error');
                }
            })
            .catch(error => {
                document.getElementById('fileName').textContent = '✗ アップロード失敗';
                document.getElementById('fileName').style.color = '#f44336';
                showMessage('アップロードエラー: ' + error, 'error');
            });
        }

        function fetchDataInfo() {
            fetch('/data_info')
            .then(response => response.json())
            .then(data => {
                if (data.count > 0) {
                    const infoDiv = document.getElementById('dataInfo');
                    infoDiv.innerHTML = `
                        現在の学習データ: <strong>${data.count}件</strong> の仕訳<br>
                        登録科目: ${data.accounts.join(', ')}
                    `;
                    infoDiv.style.display = 'block';
                }
            });
        }

        function getSuggestions() {
            const date = document.getElementById('date').value;
            const vendor = document.getElementById('vendor').value;
            const description = document.getElementById('description').value;

            if (!date || !vendor || !description) {
                showMessage('すべての項目を入力してください', 'error');
                return;
            }

            fetch('/suggest', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({vendor, description})
            })
            .then(response => response.json())
            .then(data => {
                displayResults(data.candidates);
            })
            .catch(error => {
                showMessage('エラーが発生しました: ' + error, 'error');
            });
        }

        function displayResults(candidates) {
            const resultsDiv = document.getElementById('results');
            const candidatesDiv = document.getElementById('candidates');

            if (candidates.length === 0) {
                candidatesDiv.innerHTML = '<p style="color: #999;">該当する勘定科目が見つかりませんでした</p>';
                resultsDiv.style.display = 'block';
                return;
            }

            candidatesDiv.innerHTML = '';
            candidates.forEach((candidate, index) => {
                const div = document.createElement('div');
                div.className = 'candidate';
                div.onclick = () => selectCandidate(candidate.勘定科目, div);

                const reasonsHtml = candidate.理由.map(r =>
                    `<div class="reason-item">${r}</div>`
                ).join('');

                div.innerHTML = `
                    <div class="candidate-header">
                        <span class="account-name">${candidate.勘定科目}</span>
                        <span class="score">スコア: ${candidate.スコア}</span>
                    </div>
                    <div class="reasons">${reasonsHtml}</div>
                `;

                candidatesDiv.appendChild(div);
            });

            resultsDiv.style.display = 'block';
            document.getElementById('confirmSection').style.display = 'none';
            selectedAccount = null;
        }

        function selectCandidate(account, element) {
            // 既存の選択を解除
            document.querySelectorAll('.candidate').forEach(el => {
                el.classList.remove('selected');
            });

            // 新しい選択を適用
            element.classList.add('selected');
            selectedAccount = account;

            document.getElementById('selectedAccount').textContent = account;
            document.getElementById('confirmSection').style.display = 'block';
            document.getElementById('message').style.display = 'none';
        }

        function confirmSelection() {
            if (!selectedAccount) {
                showMessage('勘定科目を選択してください', 'error');
                return;
            }

            const date = document.getElementById('date').value;
            const vendor = document.getElementById('vendor').value;
            const description = document.getElementById('description').value;
            const memo = document.getElementById('memo').value;

            fetch('/confirm', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({
                    date, vendor, description,
                    selected_account: selectedAccount,
                    memo
                })
            })
            .then(response => response.json())
            .then(data => {
                showMessage('保存しました!', 'success');
                // フォームをリセット
                setTimeout(() => {
                    document.getElementById('vendor').value = '';
                    document.getElementById('description').value = '';
                    document.getElementById('memo').value = '';
                    document.getElementById('results').style.display = 'none';
                    document.getElementById('confirmSection').style.display = 'none';
                    selectedAccount = null;
                }, 1500);
            })
            .catch(error => {
                showMessage('保存に失敗しました: ' + error, 'error');
            });
        }

        function showMessage(text, type) {
            const messageDiv = document.getElementById('message');
            messageDiv.textContent = text;
            messageDiv.className = 'message ' + type;
            messageDiv.style.display = 'block';

            if (type === 'success') {
                setTimeout(() => {
                    messageDiv.style.display = 'none';
                }, 3000);
            }
        }
    </script>
</body>
</html>
"""

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/suggest', methods=['POST'])
def suggest():
    data = request.json
    vendor = data.get('vendor', '')
    description = data.get('description', '')

    candidates = engine.suggest_accounts(vendor, description, top_k=3)

    return jsonify({'candidates': candidates})

@app.route('/confirm', methods=['POST'])
def confirm():
    data = request.json
    save_review_log(
        data.get('date'),
        data.get('vendor'),
        data.get('description'),
        data.get('selected_account'),
        data.get('memo', '')
    )
    return jsonify({'status': 'success'})

@app.route('/upload_journal', methods=['POST'])
def upload_journal():
    """過去仕訳CSVをアップロード"""
    try:
        if 'file' not in request.files:
            return jsonify({'status': 'error', 'message': 'ファイルが選択されていません'}), 400

        file = request.files['file']
        if file.filename == '':
            return jsonify({'status': 'error', 'message': 'ファイルが選択されていません'}), 400

        if not file.filename.endswith('.csv'):
            return jsonify({'status': 'error', 'message': 'CSVファイルを選択してください'}), 400

        # ファイルを保存
        file.save('journal.csv')

        # エンジンを再読み込み
        global engine
        engine.load_journal_data('journal.csv')

        return jsonify({
            'status': 'success',
            'message': 'アップロード完了',
            'count': len(engine.journal_df)
        })
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

@app.route('/data_info', methods=['GET'])
def data_info():
    """現在の学習データ情報を取得"""
    if engine.journal_df is None or len(engine.journal_df) == 0:
        return jsonify({'count': 0, 'accounts': []})

    accounts = engine.journal_df['勘定科目'].unique().tolist()
    return jsonify({
        'count': len(engine.journal_df),
        'accounts': accounts
    })

def run_flask():
    """Flaskサーバーを起動"""
    app.run(host='0.0.0.0', port=5000, debug=False)

# メイン実行部分
if __name__ == '__main__':
    print("=" * 60)
    print("勘定科目判断支援ツール起動中...")
    print("=" * 60)

    # エンジンの初期化
    engine = AccountSuggestionEngine('journal.csv')

    # Flaskサーバーをバックグラウンドで起動
    thread = Thread(target=run_flask)
    thread.daemon = True
    thread.start()

    # Google Colab用の公開URL取得
    try:
        from google.colab.output import eval_js
        print("\n✓ Google Colab環境を検出しました")
        print("✓ 以下のURLでアクセスできます:")
        print(eval_js("google.colab.kernel.proxyPort(5000)"))
    except:
        print("\n✓ ローカル環境で起動しました")
        print("✓ ブラウザで以下のURLにアクセスしてください:")
        print("  http://localhost:5000")

    print("\n" + "=" * 60)
    print("使い方:")
    print("1. 日付、取引先、摘要を入力")
    print("2. 「勘定科目を推定」ボタンをクリック")
    print("3. 提示された候補から適切な科目を選択")
    print("4. 必要に応じてメモを記入して確定")
    print("=" * 60)

    # サーバーを維持
    try:
        thread.join()
    except KeyboardInterrupt:
        print("\n\nサーバーを停止しました")

勘定科目判断支援ツール起動中...
仕訳データ読み込み完了: 10件
 * Serving Flask app '__main__'

✓ Google Colab環境を検出しました
✓ 以下のURLでアクセスできます:
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


https://5000-m-s-357ta014copal-c.us-central1-1.prod.colab.dev

使い方:
1. 日付、取引先、摘要を入力
2. 「勘定科目を推定」ボタンをクリック
3. 提示された候補から適切な科目を選択
4. 必要に応じてメモを記入して確定
